In [1]:
import os
import pandas as pd

In [2]:
def count_maidens(df, bowler):
    maiden_overs = 0
    over_groups = df[df["bowler"] == bowler].groupby(df["overs"].astype(int))  # Group by over number
    for _, over_df in over_groups:
        #print(over_df)
        if over_df["runs_given_bool"].sum() == 0 and over_df["bowler_bowls_done"].sum()==6:  # If no runs were given in the over
            maiden_overs += 1
    return maiden_overs

In [5]:
curr_path = '/Users/ritamupadhyay/Documents/Coral/table_gen'
pdf_dest_path = 'data_commentaries/odi/pdfs'
pdf_file = os.path.join(curr_path,pdf_dest_path)

In [6]:
os.chdir(curr_path)

In [7]:
gnd_truth_table = 'data_commentaries/odi/ground_truth_tables'
all_tables = 'data_commentaries/odi/tables'
gnd_truth_files = os.path.join(curr_path,gnd_truth_table)
table_file = os.path.join(curr_path,all_tables)

In [8]:
commentaries = []
bowler_gnd_truth = []
batsman_gnd_truth = []
ref_table = []
for files in os.listdir(table_file):
    file_comp_path = os.path.join(table_file,files)
    try:
        df = pd.read_csv(file_comp_path)
    except:
        continue
    print(file_comp_path)
    start = 0 
    max_end = max(df['overs'])
    end = start+5
    while end<=max_end+5:
        df_temp = df[(df['overs']>=start)&(df['overs']<=end)]
        commentary = '\n'.join(df_temp['commentary'].tolist())
        batsman_scorecard = df_temp.groupby("batsman").agg(
            Runs=("batsman_runs", "sum"),
            Balls_Faced=("batsman_bowls_faced", "sum"),
            Fours=("batsman_fours", "sum"),
            Sixes=("batsman_sixes", "sum"),
        ).reset_index()
        batsman_scorecard['S/R'] = batsman_scorecard.apply(lambda x: round((x["Runs"] / x["Balls_Faced"]) * 100,2),axis=1)
        dismissals = df_temp[df_temp["dismissal"] != ""][["batsman", "dismissal"]].drop_duplicates().dropna()
        batsman_scorecard = batsman_scorecard.merge(dismissals, on="batsman", how="left")
        batsman_scorecard = batsman_scorecard.fillna("Not Out")

        bowler_scorecard = df_temp.groupby("bowler").agg(
        Balls=("bowler_bowls_done", "sum"),  # Convert balls to overs
        Runs_Given=("bowler_runs_given", "sum"),
        Wickets=("bowler_wickets", "sum"),
        ).reset_index()
        bowler_scorecard['Overs'] = bowler_scorecard.apply(lambda x: x['Balls']//6+0.1*(x['Balls']%6),axis=1)
        bowler_scorecard["Maidens"] = bowler_scorecard["bowler"].apply(lambda b: count_maidens(df_temp, b))
        
        bowler_scorecard_file_name = os.path.join(gnd_truth_table,files[:-4]+f"_bowler_scorecard_{end}.csv")
        batsman_scorecard_file_name = os.path.join(gnd_truth_table,files[:-4]+f"_batsman_scorecard_{end}.csv")

        batsman_scorecard.to_csv(batsman_scorecard_file_name,index=False)
        bowler_scorecard.to_csv(bowler_scorecard_file_name,index=False)

        commentaries.append(commentary)
        batsman_gnd_truth.append(batsman_scorecard_file_name)
        bowler_gnd_truth.append(bowler_scorecard_file_name)
        ref_table.append(os.path.join(all_tables,files))

        end = end+5
        '''print(bowler_scorecard)
        print(batsman_scorecard)
        print("\n\n\n")
        
        if end==22:
            break'''

/Users/ritamupadhyay/Documents/Coral/table_gen/data_commentaries/odi/tables/Ball by Ball Commentary & Live Score - ZIM vs BAN, 2nd ODI.csv
/Users/ritamupadhyay/Documents/Coral/table_gen/data_commentaries/odi/tables/Ball by Ball Commentary & Live Score - ZIM vs KENYA, 1st ODI.csv
/Users/ritamupadhyay/Documents/Coral/table_gen/data_commentaries/odi/tables/Ball by Ball Commentary & Live Score - WI vs IRE, 27th Match, Group B.csv
/Users/ritamupadhyay/Documents/Coral/table_gen/data_commentaries/odi/tables/Ball by Ball Commentary & Live Score - PAK vs NZ, 1st ODI.csv
/Users/ritamupadhyay/Documents/Coral/table_gen/data_commentaries/odi/tables/Ball by Ball Commentary & Live Score - AFG vs CAN, 7th Match.csv
/Users/ritamupadhyay/Documents/Coral/table_gen/data_commentaries/odi/tables/Ball by Ball Commentary & Live Score - ENG vs PAK, 1st ODI.csv
/Users/ritamupadhyay/Documents/Coral/table_gen/data_commentaries/odi/tables/Ball by Ball Commentary & Live Score - IND vs BAN, 2nd Match, Group A.csv
/U

In [7]:
data = {'commentaries':commentaries,
        'batsman_gnd_truth':batsman_gnd_truth,
        'bowler_gnd_truth':bowler_gnd_truth,
        'ref_table':ref_table}

df_odi_complete = pd.DataFrame(data)

In [8]:
df_odi_complete

,commentaries,batsman_gnd_truth,bowler_gnd_truth,ref_table
0,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
1,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
2,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
3,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
4,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
...,...,...,...,...
1301,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
1302,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
1303,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
1304,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...


In [9]:
for idx, row in df_odi_complete.sample(10).iterrows():
    print(row['commentaries'])
    print("\n")
    df_temp = pd.read_csv(row['batsman_gnd_truth'])
    print(df_temp)
    print("\n")
    df_temp = pd.read_csv(row['bowler_gnd_truth'])
    print(df_temp)
    print("\n\n\n")

 Rampaul to Porterfield, no run
 Rampaul to Porterfield, FOUR runs
 Rampaul to Porterfield, no run
 Rampaul to Porterfield, no run
 Rampaul to Porterfield, no run
 Rampaul to Porterfield, no run 
 Bernard to Stirling, no run
 Bernard to Stirling, FOUR runs
 Bernard to Stirling, no run
 Bernard to Stirling, no run
 Bernard to Stirling, 1 wide
 Bernard to Stirling, no run
 Bernard to Stirling, no run 
 Rampaul to Porterfield, no run
 Rampaul to Porterfield, no run
 Rampaul to Porterfield, no run
 Rampaul to Porterfield, no run
 Rampaul to Porterfield, FOUR runs
 Rampaul to Porterfield, 2 runs 
 Bernard to Stirling, no run
 Bernard to Stirling, 1 leg bye
 Bernard to Porterfield, no run
 Bernard to Porterfield, OUT  William Porterfield c Sammy b Bernard 10 (14b 2x4 0x6) SR: 71.42
 Bernard to Botha, no run
 Bernard to Botha, no run 
 Rampaul to Stirling, no run
 Rampaul to Stirling, 1 wide
 Rampaul to Stirling, no run
 Rampaul to Stirling, no run
 Rampaul to Stirling, no run
 Rampaul to Sti

In [14]:
df_odi_complete = df_odi_complete.drop_duplicates(subset=['commentaries'])

In [15]:
df_odi_complete

,commentaries,batsman_gnd_truth,bowler_gnd_truth,ref_table
0,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
1,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
2,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
3,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
4,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
...,...,...,...,...
1301,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
1302,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
1303,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...
1304,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...


In [16]:
df_odi_complete.to_csv(os.path.join(curr_path,'data_commentaries/odi',"odi_complete.csv"),index=False)

In [25]:
df_ref = pd.read_csv("data_commentaries/odi/tables/Ball by Ball Commentary & Live Score - ZIM vs KENYA, 1st ODI.csv")

In [26]:
df_ref[df_ref['batsman']=='Obuya']

,raw_data,overs,runs,team_runs,commentary,bowler,batsman,batsman_runs,batsman_fours,batsman_sixes,batsman_bowls_faced,bowler_bowls_done,bowler_runs_given,bowler_wickets,dismissal,runs_given_bool
0,"0.1 \n• \nChigumbura to Obuya, no run",0.1,0,0.0,"Chigumbura to Obuya, no run",Chigumbura,Obuya,0.0,0,0,1,1,0.0,0,NaN,0
1,"0.2 \n• \nChigumbura to Obuya, no run",0.2,0,0.0,"Chigumbura to Obuya, no run",Chigumbura,Obuya,0.0,0,0,1,1,0.0,0,NaN,0
2,"0.3 \n1 \nChigumbura to Obuya, 1 run",0.3,1,1.0,"Chigumbura to Obuya, 1 run",Chigumbura,Obuya,1.0,0,0,1,1,1.0,0,NaN,1
6,"1.1 \n2 \nJarvis to Obuya, 2 runs",1.1,2,2.0,"Jarvis to Obuya, 2 runs",Jarvis,Obuya,2.0,0,0,1,1,2.0,0,NaN,1
7,"1.2 \n1 \nJarvis to Obuya, 1 run",1.2,1,1.0,"Jarvis to Obuya, 1 run",Jarvis,Obuya,1.0,0,0,1,1,1.0,0,NaN,1
11,"1.5 \n• \nJarvis to Obuya, no run",1.5,0,0.0,"Jarvis to Obuya, no run",Jarvis,Obuya,0.0,0,0,1,1,0.0,0,NaN,0
12,"1.6 \n• \nJarvis to Obuya, no run \nend of ove...",1.6,0,0.0,"Jarvis to Obuya, no run",Jarvis,Obuya,0.0,0,0,1,1,0.0,0,NaN,0
19,"3.1 \n• \nJarvis to Obuya, no run",3.1,0,0.0,"Jarvis to Obuya, no run",Jarvis,Obuya,0.0,0,0,1,1,0.0,0,NaN,0
20,"3.2 \n• \nJarvis to Obuya, no run",3.2,0,0.0,"Jarvis to Obuya, no run",Jarvis,Obuya,0.0,0,0,1,1,0.0,0,NaN,0
21,"3.3 \n• \nJarvis to Obuya, no run",3.3,0,0.0,"Jarvis to Obuya, no run",Jarvis,Obuya,0.0,0,0,1,1,0.0,0,NaN,0
